In [ ]:
# Importing libraries
import os
import math
import random
import operator
import string
import logging
import gensim
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['want','got','say','amp', 'thi', 'ain']
stopwords.extend(newStopWords)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Cleaning the Dataset
def preprocess(text_string):
  text_string = text_string.translate(string.punctuation)
    ## Convert words to lower case and split them
  text_string = text_string.lower().split()
    ## Remove stop words
  text_string = [w for w in text_string if not w in stopwords and len(w) >= 3]
  text_string = " ".join(text_string)
  space_pattern = '\s+'
  giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
      '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
  mention_regex = '@\w+\W+'
  excliamation_regex = '!+'
  RT_regex = 'RT'
  parsed_text = re.sub(space_pattern, ' ', text_string)
  parsed_text = re.sub(excliamation_regex,'! ',parsed_text)
  parsed_text = re.sub(giant_url_regex, '', parsed_text)
  parsed_text = re.sub(mention_regex, '', parsed_text)
  parsed_text = re.sub(RT_regex,'', parsed_text)
  parsed_text = re.sub('(.)\\1{2,}', '\\1', parsed_text)
  parsed_text = re.sub('bitche', 'bitch',parsed_text)
  parsed_text.replace("#", "")
  parsed_text = parsed_text.split()
  stemmer = SnowballStemmer('english')
  stemmed_words = [stemmer.stem(word) for word in parsed_text]
  parsed_text = " ".join(stemmed_words)
  #parsed_text = parsed_text.code("utf-8", errors='ignore')
  parsed_text = re.sub('[^a-zA-Z]',' ', parsed_text)
  return parsed_text

In [ ]:
# Loading the dataset
df = pd.read_csv("https://raw.githubusercontent.com/MitHar24/dataset/main/New%20combined_Dataset.csv")
df.head()

,Label,Tweet
0,1,@BDUTT Shame on your hate news n fake news.
1,1,@AP Pita can suck my ass! What a shit show of ...
2,0,@simonjames67 @Mr_John_Harvey_ @theJeremyVine ...
3,0,@GMB @SeanFletcherTV Oh wow.........what the h...
4,0,@KallipolisState @talkRADIO @JuliaHB1 @spikedo...


In [ ]:
# Printing the Processed Dataset
df['processed'] = df['Tweet'].apply(lambda x: preprocess(x))
df.head()

,Label,Tweet,processed
0,1,@BDUTT Shame on your hate news n fake news.,shame hate news fake news
1,1,@AP Pita can suck my ass! What a shit show of ...,pita suck ass shit show organ put anim save
2,0,@simonjames67 @Mr_John_Harvey_ @theJeremyVine ...,mr john harvey waitros i m littl worri bank b...
3,0,@GMB @SeanFletcherTV Oh wow.........what the h...,seanfletchertv wow what hell know wolv relega...
4,0,@KallipolisState @talkRADIO @JuliaHB1 @spikedo...,talkradio spikedonlin noth like outrag now ma...


In [ ]:
# Loading Pre-trained Gensim model
!pip install wget

In [ ]:
import wget
wget.download("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz")

'GoogleNews-vectors-negative300.bin (3).gz'

In [ ]:
# Applying Word2Vec model
from gensim.models import Word2Vec
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
wv = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
wv.init_sims(replace = True)

In [ ]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [ ]:
# Splitting the training ans testing data
X = df['processed']
Y = df['Label']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 0)

test_tokenized = x_test.apply(lambda x: w2v_tokenize_text(x)).values
train_tokenized = x_train.apply(lambda x: w2v_tokenize_text(x)).values

X_train = word_averaging_list(wv,train_tokenized)
X_test = word_averaging_list(wv,test_tokenized)

In [ ]:
# Scaling the X parameters i.e. X_train and X_test
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

# # Scaling the data
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.fit_transform(X_test)

In [ ]:
def reduce_features(solution, features):
    selected_elements_indices = numpy.where(solution == 1)[0]
    reduced_features = features[:, selected_elements_indices]
    return reduced_features

In [ ]:
def sigmoid(x):
  return (1/(1+np.exp(-1*x)))

In [ ]:
from time import time
def init(X, y, popu_size, size, model,val_x,particles_init=None):
  particles = np.random.randint(2, size=(popu_size,size))
  if particles_init is not None:
    particles = particles_init

  velocities = np.random.rand(popu_size,size)
  pbest = np.zeros((popu_size,size))
  pbest_score = np.ones((popu_size,),dtype=float)*0
  model.fit(X,y)
  sc = model.score(val_x,y_test)
  gbest_score = sc
  gbest = np.ones((size,),dtype=float)
  gbest_popu = np.ones((popu_size,size))
  return particles,velocities,pbest,gbest,pbest_score,gbest_score,gbest_popu

In [ ]:
def check_fitness(X,Y,particles,pbest,gbest,pbest_score,gbest_score,popu_size,model,size,val_x,gbest_popu):
  best_itr = np.zeros((size,))
  best_itr_sc = 0
  for i in range(popu_size):
    # Error
    # print(particles[i].shape)
    X_togive = np.multiply(particles[i], X)
    test_d = np.multiply(particles[i], val_x)
    model.fit(X_togive, Y)
    score = model.score(test_d,y_test)
    
    if score > pbest_score[i]:
      pbest_score[i] = score
      pbest[i] = particles[i]
    
    if score > gbest_score:
      gbest_score = score
      gbest = pbest[i]
      gbest_popu = pbest
    
    if score >best_itr_sc:
      best_itr_sc = score
      best_itr = particles[i]
  return pbest,gbest,pbest_score,gbest_score,best_itr_sc,best_itr

In [ ]:
def update_particle(velocities,particles,pbest,gbest,popu_size,size,itr,maxItr,best_itr):
  wmax = 1.0
  wmin = 0.6
  kmin = 1.5
  kmax = 4.0
  w = wmax-(itr/maxItr)*(wmax-wmin)
  k = kmin+(itr/maxItr)*(wmax-wmin)
  for i in range(popu_size):
    velocities[i] = w*velocities[i] + k*np.random.rand()*(pbest[i]-particles[i]) + k*np.random.rand()*(gbest-particles[i])+ k*np.random.rand()*(best_itr-particles[i])
    for kk in range(size):
      if velocities[i][kk]>4.0:
        velocities[i][kk] = 4.0
      elif velocities[i][kk]<-4.0:
        velocities[i][kk] = -4.0
    velo_sig = sigmoid(velocities[i])
    for kk in range(size):
        particles[i][kk] = (np.random.randn()<velo_sig[kk])*1
  return particles

In [ ]:
def pso(X, Y, popu_size, model, maxItr, size,val_x,particles_init=None):
  all_scores = []
  particles,velocities,pbest,gbest,pbest_score,gbest_score,gbest_popu = init(X,Y,popu_size,size,model,val_x,particles_init)
  itr = 0
  early_stop = 0
  while(itr < maxItr):
    start = time()
    pbest,gbest,pbest_score,gbest_score,best_itr_sc,best_itr = check_fitness(X,Y,particles,pbest,gbest,pbest_score,gbest_score,popu_size,model,size,val_x,gbest_popu)
    particles = update_particle(velocities,particles,pbest,gbest,popu_size,size,itr,maxItr,best_itr)
    end = time()
    if all_scores:
      if gbest_score==all_scores[-1]:
        early_stop+=1
        if early_stop==35:
          print("Early Stopping due to no increase in fitness...")
          return gbest,gbest_popu,all_scores
      else:
        early_stop = 0
    print("Epochs:",itr+1,"Time Taken:",round((end - start), 2),"secs","Best Score:",gbest_score,"Best Score Itr",best_itr_sc)
    all_scores.append(gbest_score)
    itr+=1
  print("Best Score ",gbest_score)
  print("Best Features: ",gbest)
  gbest_indices = np.where(gbest == 1)[0]
  gbest_num_elements = gbest_indices.shape[0]
  print("Indices Selected: ",gbest_indices)
  print("Total Elements Selected: ",gbest_num_elements)
  return gbest,gbest_popu,all_scores

In [ ]:
from sklearn.ensemble import RandomForestClassifier
lr = RandomForestClassifier(n_estimators = 100)
new_x,best_popu,all_scores = pso(X_train,y_train,8,lr,5,300,X_test,None)

Epochs: 1 Time Taken: 552.39 secs Best Score: 0.7594230557492568 Best Score Itr 0.7586890299849525
Epochs: 2 Time Taken: 787.92 secs Best Score: 0.7595698609021176 Best Score Itr 0.7595698609021176
Epochs: 3 Time Taken: 795.82 secs Best Score: 0.7615884317539545 Best Score Itr 0.7615884317539545
Epochs: 4 Time Taken: 754.39 secs Best Score: 0.7615884317539545 Best Score Itr 0.7607443021250045
Epochs: 5 Time Taken: 801.44 secs Best Score: 0.7630197819943481 Best Score Itr 0.7630197819943481
Best Score  0.7630197819943481
Best Features:  [1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1

In [ ]:
x_to_train = np.multiply(new_x,X_train)
xTest = np.multiply(new_x,X_test)

In [ ]:
def reduce_features(solution, features):
    selected_elements_indices = numpy.where(solution == 1)[0]
    reduced_features = features[:, selected_elements_indices]
    return reduced_features

In [ ]:
def cal_pop_fitness(pop, features, labels):#, train_indices, test_indices):
    accuracies = numpy.zeros(pop.shape[0])
    idx = 0
    for curr_solution in pop:
        reduced_features = reduce_features(curr_solution, x_to_train)
        reduced_features_t = reduce_features(curr_solution, xTest)
        #train_data = reduced_features[train_indices, :]
        #test_data = reduced_features[test_indices, :]

        #train_labels = labels[train_indices]
        #test_labels = labels[test_indices]
        # train_data,test_data,train_labels,test_labels = train_test_split(reduced_features,labels, test_size=0.05, random_state=42)
        #SV_classifier = SVC(kernel='rbf')
        SV_classifier = RandomForestClassifier(n_estimators = 100)
        SV_classifier.fit(X=reduced_features, y=y_train)
        #predictions = SV_classifier.predict(test_data)
        accuracies[idx] = SV_classifier.score(reduced_features_t,y_test)
        idx = idx + 1
    return accuracies

In [ ]:
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

In [ ]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring


In [ ]:
def mutation(offspring_crossover, num_mutations=2):
    mutation_idx = numpy.random.randint(low=0, high=offspring_crossover.shape[1], size=num_mutations)
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        offspring_crossover[idx, mutation_idx] = 1 - offspring_crossover[idx, mutation_idx]
    return offspring_crossover

In [ ]:
import numpy
import pickle
import matplotlib.pyplot
data_inputs = x_to_train
data_outputs = xTest
num_samples = data_inputs.shape[0]
num_feature_elements = data_inputs.shape[1]

In [ ]:
sol_per_pop = 8 # Population size.
num_parents_mating = 4 # Number of parents inside the mating pool.
num_mutations = 3 # Number of elements to mutate.
# Defining the population shape.
pop_shape = (sol_per_pop, num_feature_elements)
# Creating the initial population.
new_population = best_popu
print(new_population.shape)
best_outputs = []

(8, 300)


In [ ]:
num_generations = 5
for generation in range(num_generations):
    #print("Generation : ", generation)
    start = time()
    # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population, data_inputs, data_outputs)#, train_indices, test_indices)
    best_outputs.append(numpy.max(fitness))
    # The best result in the current iteration.
    #print("Best result : ", best_outputs[-1])
    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, num_parents_mating)
    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, offspring_size=(pop_shape[0]-parents.shape[0], num_feature_elements))
    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover, num_mutations=num_mutations)
    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    end = time()
    best_match_idx = numpy.where(fitness == numpy.max(fitness))[0]
    best_match_idx = best_match_idx[0]
    best_solution = new_population[best_match_idx, :]
    print("Epochs:",generation,"Time Taken:",(end-start),"secs","Best result : ", best_outputs[-1])
# Getting the best solution after iterating finishing all generations.
# At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population, data_inputs, data_outputs)#,train_indices, test_indices)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))[0]
best_match_idx = best_match_idx[0]
best_solution = new_population[best_match_idx, :]
best_solution_indices = numpy.where(best_solution == 1)[0]
best_solution_num_elements = best_solution_indices.shape[0]
best_solution_fitness = fitness[best_match_idx]

Epochs: 0 Time Taken: 799.9246525764465 secs Best result :  0.7627628729768415
Epochs: 1 Time Taken: 801.8285043239594 secs Best result :  0.7621389510771828
Epochs: 2 Time Taken: 790.3625564575195 secs Best result :  0.761331522736448
Epochs: 3 Time Taken: 788.9676730632782 secs Best result :  0.7615517304657393
Epochs: 4 Time Taken: 798.1072866916656 secs Best result :  0.7622857562300437


In [ ]:
fitness = cal_pop_fitness(new_population, data_inputs, data_outputs)#,train_indices, test_indices)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))[0]
best_match_idx = best_match_idx[0]
best_solution = new_population[best_match_idx, :]
best_solution_indices = numpy.where(best_solution == 1)[0]
best_solution_num_elements = best_solution_indices.shape[0]
best_solution_fitness = fitness[best_match_idx]

In [ ]:
print("best_match_idx : ", best_match_idx)
print("best_solution : ", best_solution)
print("Selected indices : ", best_solution_indices)
print("Number of selected elements : ", best_solution_num_elements)
print("Best solution fitness : ", best_solution_fitness)

best_match_idx :  2
best_solution :  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
Selected indices :  [  0   1   2   3   4   5   6 

In [ ]:
scores = all_scores + best_outputs

In [ ]:
X_temp = X.apply(lambda x: w2v_tokenize_text(x)).values
X = word_averaging_list(wv,X_temp)

In [ ]:
best_solution
X_new = np.multiply(X,best_solution)

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_labels,test_labels = train_test_split(X_new,Y, test_size = 0.33, random_state = 0)



# # Scaling the data
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler()
# train_data = sc.fit_transform(train_data)
# test_data = sc.fit_transform(test_data)

# Scaling the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_data = sc.fit_transform(train_data)
test_data = sc.fit_transform(test_data)


classifier = RandomForestClassifier(n_estimators = 100)
classifier.fit(train_data,train_labels)
Y_pred = classifier.predict(test_data)
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
Y_test = test_labels
f1 = f1_score(Y_test, Y_pred)
ps = precision_score(Y_test, Y_pred)
rs = recall_score(Y_test, Y_pred)
asc = accuracy_score(Y_test, Y_pred)
print(f"F1 Score {f1}")
print(f"Precesion Score {ps}")
print(f"Recall Score {rs}")
print(f"Accuracy Score {asc}")

F1 Score 0.7049769112365316
Precesion Score 0.8357664233576643
Recall Score 0.6095829636202307
Accuracy Score 0.7678643520387566
